In [8]:
import os
os.chdir('..')
from src.model.HMM.HMM import HMM
from src.model.HMM.data import train_test_split, get_dataset

In [10]:
exe_path = '/Users/yikaimao/Desktop/DSC_180B/Intel-capstone/data/processed/exe.csv'
X, y = get_dataset(exe_path)
X_train, y_train, X_test, y_test = train_test_split(X, y)
model = HMM()
model.fit(X_train, y_train)

In [28]:
sorted(model.transition_matrix.items(), key=lambda x: list(x[1].values())[0], reverse=True)

[('vpndownloader.exe',
  defaultdict(float,
              {'firefox.exe': 1.0,
               '<s>': 0.0,
               'StartMenuExperienceHost.exe': 0.0,
               'Microsoft.AAD.BrokerPlugin.exe': 0.0,
               'Taskmgr.exe': 0.0,
               'iCloudServices.exe': 0.0,
               'iCloud.exe': 0.0,
               'ApplicationFrameHost.exe': 0.0,
               'WindowsCamera.exe': 0.0,
               'ConsoleWindowClass': 0.0,
               'SystemSettings.exe': 0.0,
               '#32770': 0.0,
               'DirectUIHWND': 0.0,
               '#32768': 0.0,
               'Button': 0.0,
               'Windows.UI.Core.CoreWindow': 0.0,
               'ToolbarWindow32': 0.0,
               'Microsoft.Photos.exe': 0.0,
               'RiotClientServices.exe': 0.0,
               'MSTaskListWClass': 0.0,
               'RiotWindowClass': 0.0,
               'Installer.exe': 0.0,
               'AMDBugReportForm.exe': 0.0,
               'WechatBrowser.exe': 0.0,

In [19]:
import pandas as pd
transition = pd.DataFrame(model.transition_matrix).T
transition.head(10)

,Teams.exe,explorer.exe,firefox.exe,Discord.exe,chrome.exe,SearchApp.exe,VsDebugConsole.exe,Shell_TrayWnd,ApplicationManager_DesktopShellWindow,RiotClientUx.exe,...,slack.exe,WeChat.exe,LeagueClient.exe,LeagueCrashHandler.exe,audacity.exe,7zFM.exe,DB Browser for SQLite.exe,MSTaskListWClass,MozillaWindowClass,WerFault.exe
<s>,0.285714,0.178571,0.142857,0.142857,0.071429,0.035714,0.035714,0.035714,0.035714,0.035714,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
VsDebugConsole.exe,0.000000,0.176471,0.352941,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
firefox.exe,0.008716,0.156894,0.000000,0.030111,0.028526,0.004754,0.005547,0.000000,0.000000,0.013471,...,0.000000,0.012678,0.0,0.0,0.006339,0.000000,0.000000,0.0,0.0,0.000792
explorer.exe,0.034682,0.000000,0.169557,0.044316,0.026975,0.057803,0.003854,0.000000,0.000000,0.025048,...,0.003854,0.019268,0.0,0.0,0.005780,0.007707,0.009634,0.0,0.0,0.001927
Teams.exe,0.000000,0.269231,0.064103,0.102564,0.397436,0.012821,0.000000,0.000000,0.000000,0.012821,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
SearchApp.exe,0.021277,0.382979,0.148936,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,...,0.000000,0.021277,0.0,0.0,0.021277,0.000000,0.000000,0.0,0.0,0.000000
LeagueClientUx.exe,0.001393,0.064067,0.823120,0.005571,0.000000,0.000000,0.000000,0.000000,0.000000,0.012535,...,0.000000,0.009749,0.0,0.0,0.008357,0.000000,0.000000,0.0,0.0,0.001393
Discord.exe,0.027682,0.062284,0.169550,0.000000,0.564014,0.000000,0.000000,0.000000,0.000000,0.006920,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
notepad.exe,0.000000,0.250000,0.250000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
League of Legends.exe,0.000000,0.085511,0.650831,0.002375,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.016627,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000


### Top-1 accuracy

In [3]:
pred_1 = model.predict(X_test)
accuracy_1 = model.evaluate(pred_1, y_test)
accuracy_1

0.474059003051882

### Top-2 accuracy

In [4]:
pred_2 = model.predict(X_test, top=2)
accuracy_2 = model.evaluate(pred_2, y_test)
accuracy_2

0.6581892166836215

### Top-3 accuracy

In [5]:
pred_3 = model.predict(X_test, top=3)
accuracy_3 = model.evaluate(pred_3, y_test)
accuracy_3

0.7589013224821973